In [1]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, GRU, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#importing the dataset
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_csv('ETH-USD (2).csv')

In [ ]:
df

In [ ]:
#Get number of rows and colums in the data set
df.shape

In [ ]:
#Create a new dataframe with only the 'Close column'
data = df.filter(['Close'])

#Convert the dataframe to a numpy array
dataset = data.values

#Get the number of rows to train the model
from sklearn.model_selection import train_test_split

# Splitting the dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Printing the sizes of the split datasets
print("Training data size:", len(train_data))
print("Testing data size:", len(test_data))

In [ ]:
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
#Create the training dataset
#Create the scaled training dataset

train_data = scaled_data[0:training_data_len , :]

#split the into x_train and y_train dataset
x_train = []
y_train = []

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<= 61:
    print(x_train)
    print(y_train)
    print()

In [ ]:
#Convert the x_train and y_train to numpy arrays so we can use them in training our models
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the data to fit model into:
#Number of samples, Number of timestamps, Number of features(Close since we are looking for closing price of data)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#Build the LSTM+GRU Model
model = Sequential()
model.add(LSTM(50, return_sequences=True,input_shape=(x_train.shape[1], 1)))
model.add(GRU(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=100)

In [ ]:
#Create Testing dataset
#Create a new array containing scaled values from index 1568 to 2034
test_data = scaled_data[training_data_len - 60:, :]
#Create the datasets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
 x_test.append(test_data[i-60:i, 0])
 

In [ ]:
#Convert data to numpy array
x_test = np.array(x_test)

In [ ]:
#Reshape the Test dataset
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the models predicted price values
predictions = model.predict(x_test)
#Unscaling values and making x_test and y_test have same values
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Evaluate model
#Get the RMSE (root mean square error)
rmse = np.sqrt( np.mean( predictions - y_test)**2 )
rmse

In [ ]:
#Get the Mean Absolute Error (MAE)
mae = np.mean(np.abs(predictions - y_test))
mae

In [ ]:
#Get the Mean Squared Error (MSE)
mse = np.mean((predictions - y_test) ** 2)
mse

In [ ]:
#Get the R-squared (R2) Score
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
r2

In [ ]:
#Get the Explained Variance Score
from sklearn.metrics import explained_variance_score
evs = explained_variance_score(y_test, predictions)
evs

In [ ]:
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Visualize the data
plt.figure(figsize=(10,5))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val','Predictions'], loc='lower right')

In [ ]:
#Show the valid(actual closing price) and predicted prices
valid

In [ ]:
#Try and predict price of June 6th 2023
#Import dataset
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_csv('ETH-USD (2).csv')

In [ ]:
#Predict the price of June 6th 2023
#create new dataframe
new_df = df.filter(['Close'])
#Get the last 60 days closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
#Import price of June 6th 2023
from google.colab import files
uploaded=files.upload()

In [ ]:
df = pd.read_csv('June 6th 2023.csv')

In [ ]:
#Print Closing Price of June 6th 2023
print(df['Close'])